In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pmdarima import auto_arima
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae
import pmdarima as pm
import pandas.util.testing as tm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot

C:\Users\ygorf\AppData\Local\Temp\ipykernel_11184\1586935810.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
df_test = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')
df_ss = pd.read_csv('sample_submission.csv')

c:\Users\ygorf\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_train.shape

(1300980, 76)

In [19]:
df_train.iloc[:20,:20]

,SEM_NOT,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,CS_ZONA,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE
0,38,37,RS,005 CRS,1608.0,CAXIAS DO SUL,430510,M,73,3,3073,6,1,NaN,RS,2.0,NaN,2.0,2.0,NaN
1,46,46,MS,NRS DE PONTA PORA,1972.0,PONTA PORA,500660,M,27,3,3027,6,1,4.0,MS,1.0,NaN,NaN,NaN,1.0
2,18,17,CE,1 CRES FORTALEZA,1519.0,FORTALEZA,230440,F,2,3,3002,6,4,5.0,CE,1.0,NaN,2.0,2.0,1.0
3,4,3,SP,GVE VII SANTO ANDRE,1332.0,MAUA,352940,F,71,3,3071,5,4,NaN,SP,1.0,NaN,2.0,2.0,2.0
4,12,11,MG,JANUARIA,1470.0,SAO JOAO DA PONTE,316240,F,91,3,3091,5,4,0.0,MG,2.0,NaN,2.0,2.0,2.0
5,10,10,MG,BELO HORIZONTE,1449.0,BELO HORIZONTE,310620,F,78,3,3078,5,4,9.0,MG,1.0,NaN,9.0,9.0,2.0
6,7,7,BA,NUCLEO REGIONAL DE SAUDE EXTREMO SUL,1388.0,ITABELA,291465,M,92,3,3092,6,4,0.0,BA,1.0,NaN,2.0,2.0,2.0
7,2,2,RJ,NaN,NaN,RIO DE JANEIRO,330455,M,1,3,3001,6,4,5.0,RJ,1.0,2.0,2.0,2.0,1.0
8,7,7,MG,DIAMANTINA,1450.0,DIAMANTINA,312160,F,87,3,3087,9,9,NaN,MG,1.0,9.0,2.0,9.0,2.0
9,28,27,SP,GVE X OSASCO,1335.0,COTIA,351300,F,68,3,3068,5,4,9.0,SP,1.0,NaN,2.0,9.0,2.0


In [7]:
print(df_train.columns)

Index(['SEM_NOT', 'SEM_PRI', 'SG_UF_NOT', 'ID_REGIONA', 'CO_REGIONA',
       'ID_MUNICIP', 'CO_MUN_NOT', 'CS_SEXO', 'NU_IDADE_N', 'TP_IDADE',
       'COD_IDADE', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF', 'CS_ZONA',
       'SURTO_SG', 'NOSOCOMIAL', 'AVE_SUINO', 'FEBRE', 'TOSSE', 'GARGANTA',
       'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_SIN',
       'OUTRO_DES', 'PUERPERA', 'FATOR_RISC', 'CARDIOPATI', 'HEMATOLOGI',
       'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI',
       'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OBES_IMC', 'OUT_MORBI',
       'MORB_DESC', 'VACINA', 'MAE_VAC', 'M_AMAMENTA', 'ANTIVIRAL',
       'TP_ANTIVIR', 'HOSPITAL', 'UTI', 'SUPORT_VEN', 'RAIOX_RES', 'RAIOX_OUT',
       'AMOSTRA', 'TP_AMOSTRA', 'OUT_AMOST', 'CLASSI_FIN', 'HISTO_VGM',
       'PAC_COCBO', 'PAC_DSCBO', 'OUT_ANIM', 'DOR_ABD', 'FADIGA', 'PERD_OLFT',
       'PERD_PALA', 'TOMO_RES', 'TOMO_OUT', 'VACINA_COV', 'DOSE_1_COV',
       'DOSE_2_COV', 'DOSE_REF